In [51]:
import dask.dataframe as dd
import cmd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [52]:
data = dd.read_csv("../data/raw/train_ver2.csv",dtype={'age': object,
                        'antiguedad': object,
                        'cod_prov': float,
                        'ind_actividad_cliente': float,
                        'ind_nom_pens_ult1': float,
                        'ind_nomina_ult1': float,
                        'ind_nuevo': float,
                        'indrel': float,
                        'tipodom': float,
                        'ult_fec_cli_1t': object,
                        'conyuemp': object,
                        'indrel_1mes': object })


In [53]:
data.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [54]:
print('Columns in the train data are \n')
columns = list(data.columns)
cmd.Cmd().columnize(columns, displaywidth=80)

Columns in the train data are 

fecha_dato       tiprel_1mes            ind_ahor_fin_ult1  ind_ecue_fin_ult1
ncodpers         indresi                ind_aval_fin_ult1  ind_fond_fin_ult1
ind_empleado     indext                 ind_cco_fin_ult1   ind_hip_fin_ult1 
pais_residencia  conyuemp               ind_cder_fin_ult1  ind_plan_fin_ult1
sexo             canal_entrada          ind_cno_fin_ult1   ind_pres_fin_ult1
age              indfall                ind_ctju_fin_ult1  ind_reca_fin_ult1
fecha_alta       tipodom                ind_ctma_fin_ult1  ind_tjcr_fin_ult1
ind_nuevo        cod_prov               ind_ctop_fin_ult1  ind_valo_fin_ult1
antiguedad       nomprov                ind_ctpp_fin_ult1  ind_viv_fin_ult1 
indrel           ind_actividad_cliente  ind_deco_fin_ult1  ind_nomina_ult1  
ult_fec_cli_1t   renta                  ind_deme_fin_ult1  ind_nom_pens_ult1
indrel_1mes      segmento               ind_dela_fin_ult1  ind_recibo_ult1  


In [55]:
customers = data.ncodpers.unique()
print('Number of data points in train:', len(df))
print('Number of Unique Customer in train set:', len(customers.compute()))

Number of data points in train: 13647309
Number of Unique Customer in train set: 956645


In [56]:
data.shape

(Delayed('int-a0a4d973-4570-4b92-bd24-63d84cd28bfc'), 48)

In [57]:
data.isnull().sum().compute()

fecha_dato                      0
ncodpers                        0
ind_empleado                27734
pais_residencia             27734
sexo                        27804
age                             0
fecha_alta                  27734
ind_nuevo                   27734
antiguedad                      0
indrel                      27734
ult_fec_cli_1t           13622516
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
ind_ahor_fin_ult1               0
ind_aval_fin_ult1               0
ind_cco_fin_ult1                0
ind_cder_fin_ult1               0
ind_cno_fin_ult1                0
ind_ctju_fin_u

In [58]:
def preprocess_missing_values(data, null_columns, target_columns):

  for column in null_columns:
    if column not in target_columns:
      print('preprocessing', column)
      custs = data.cust_code[data[column].isna()].unique().compute()
      print('Number of customers whose {} data is missing is {}'.format(column, custs.shape[0]))
      available_custs = data.cust_code[(~data[column].isna()) & (data.cust_code.isin(custs))].unique().compute()
      print('Number of customers whose data is present elsewhere is {}'.format(available_custs.shape[0]))

      #Creating a dummy data set as we can not assign values to dask dataframes
      dummy = data[data.cust_code.isin(available_custs)].compute()
      data = data[~data.cust_code.isin(available_custs)]

      available_values = dummy[(dummy.cust_code.isin(available_custs))&(~dummy[column].isna())].groupby(by='cust_code')[column].first()
      available_dict = dict(zip(available_values.index, available_values.values))
      dummy[column][(dummy.cust_code.isin(available_custs))&(dummy[column].isna())] = dummy['cust_code'][(dummy.cust_code.isin(available_custs))&(dummy[column].isna())].map(available_dict)
      print('processed', column)
      data = dd.concat([data, dd.from_pandas(dummy, npartitions = 20)])
      print('-'*100)
    else:
      print('preprocessing', column)
      print('filling na values with 0')

      dummy = data[data[column].isna()].compute()
      dummy[column] = 0
      data = data[~data[column].isna()]
      data = dd.concat([data, dd.from_pandas(dummy, npartitions = 20)])
      print('processed', column)
      print('-'*100)
  
  return data


In [59]:
#Target columns or products
target_columns = ['savings_account', 'guarentees', 'current_account', 'derivative_account', 'payroll_account', 'junior_account', 'mas_account',
'perticular_account', 'perticular_plus', 'st_deposit', 'mt_deposits', 'lt_deposits', 'e_account', 'funds', 'mortgage',
 'pension', 'loan', 'tax', 'credit_card', 'securities', 'home_account', 'payroll', 'pension2', 'direct_debit']

user_features = ['fetch_date', 'cust_code', 'emp_index', 'country', 'sex', 'age', 'cust_date', 'new_cust', 'cust_seniority',
'indrel', 'last_date_as_primary', 'cust_type', 'cust_rel', 'residence_index', 'foreigner_index', 'spouse_index',
'joining_channel', 'deceased', 'address_type', 'prov_code','prov_name', 'activity_index', 'income', 'segmentation']


In [61]:
null_columns = data.isna().sum()[data.isna().sum()>0].index.compute()
data = preprocess_missing_values(data, null_columns, target_columns)

preprocessing ind_empleado


AttributeError: 'DataFrame' object has no attribute 'cust_code'